In [265]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import math

In [266]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [267]:
Invoice = pd.read_excel("Company X - Order Report.xlsx")
Zone = pd.read_excel("Company X - Pincode Zones.xlsx")
SKU = pd.read_excel("Company X - SKU Master.xlsx")
Courier_invoice = pd.read_excel("Courier Company - Invoice.xlsx")
Courier_rate = pd.read_excel("Courier Company - Rates.xlsx")
Result = pd.read_excel("Expected_Result.xlsx")

## Count

In [268]:
print(Invoice['ExternOrderNo'].count())
print(Zone['Zone'].count())
print(SKU['SKU'].count())
print(Courier_invoice['AWB Code'].count())
print(Courier_rate['fwd_a_fixed'].count())

400
124
66
124
1


## Columns

In [269]:
Invoice.columns

Index(['ExternOrderNo', 'SKU', 'Order Qty'], dtype='object')

In [270]:
Zone.columns

Index(['Warehouse Pincode', 'Customer Pincode', 'Zone'], dtype='object')

In [271]:
SKU.columns

Index(['SKU', 'Weight (g)'], dtype='object')

In [272]:
Courier_invoice.columns

Index(['AWB Code', 'Order ID', 'Charged Weight', 'Warehouse Pincode',
       'Customer Pincode', 'Zone', 'Type of Shipment', 'Billing Amount (Rs.)'],
      dtype='object')

In [273]:
Courier_rate.columns

Index(['fwd_a_fixed', 'fwd_a_additional', 'fwd_b_fixed', 'fwd_b_additional',
       'fwd_c_fixed', 'fwd_c_additional', 'fwd_d_fixed', 'fwd_d_additional',
       'fwd_e_fixed', 'fwd_e_additional', 'rto_a_fixed', 'rto_a_additional',
       'rto_b_fixed', 'rto_b_additional', 'rto_c_fixed', 'rto_c_additional',
       'rto_d_fixed', 'rto_d_additional', 'rto_e_fixed', 'rto_e_additional'],
      dtype='object')

In [274]:
Result.columns

Index(['Index', 'Count', 'Amount'], dtype='object')

## Total Amount

In [275]:
np.sum(Courier_invoice['Billing Amount (Rs.)'])

np.float64(13648.2)

## Remove Duplicates Row

In [276]:
Invoice = Invoice.drop_duplicates()
Zone = Zone.drop_duplicates()
SKU = SKU.drop_duplicates()
Courier_invoice = Courier_invoice.drop_duplicates()
Courier_rate = Courier_rate.drop_duplicates()

## Upper case zone

In [277]:
Zone

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
114,121003,324008,b
115,121003,302020,b
119,121003,325207,b
120,121003,303702,b


In [278]:
print(Invoice['ExternOrderNo'].count())
print(Zone['Zone'].count())
print(SKU['SKU'].count())
print(Courier_invoice['AWB Code'].count())
print(Courier_rate['fwd_a_fixed'].count())

398
108
65
124
1


## Merge

In [279]:
#Rename
Invoice.rename(columns = {'ExternOrderNo' : 'Order ID'})
Invoice.head(1)

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0


In [280]:
Invoice_SKU = pd.merge(Invoice, SKU, on='SKU', how='inner')
Invoice_SKU.head(2)

,ExternOrderNo,SKU,Order Qty,Weight (g)
0,2001827036,8904223818706,1.0,127
1,2001827036,8904223819093,1.0,150


In [281]:
Invoice_SKU['Weight(kg)'] = (Invoice_SKU['Weight (g)'] * Invoice_SKU['Order Qty'])/ 1000
Invoice_SKU['Weight Slab as per X'] = Invoice_SKU['Weight(kg)'].apply(lambda x: math.ceil(x * 2) / 2)
Invoice_SKU.head(3)

,ExternOrderNo,SKU,Order Qty,Weight (g),Weight(kg),Weight Slab as per X
0,2001827036,8904223818706,1.0,127,0.127,0.5
1,2001827036,8904223819093,1.0,150,0.150,0.5
2,2001827036,8904223819109,1.0,100,0.100,0.5


In [288]:
Invoice_SKU = Invoice_SKU.rename(columns={'ExternOrderNo': 'Order ID'})

In [289]:
Invoice_SKU_Courier_invoice = pd.merge(Invoice_SKU, Courier_invoice, how='inner', on='Order ID')
Invoice_SKU_Courier_invoice.head(2)

,Order ID,SKU,Order Qty,Weight (g),Weight(kg),Weight Slab as per X,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.)
0,2001827036,8904223818706,1.0,127,0.127,0.5,1091122418320,1.6,121003,173213,b,Forward charges,117.9
1,2001827036,8904223819093,1.0,150,0.150,0.5,1091122418320,1.6,121003,173213,b,Forward charges,117.9


In [290]:
grouped = Invoice_SKU_Courier_invoice.groupby('Type of Shipment')['Weight(kg)'].mean()
grouped 

Type of Shipment
Forward and RTO charges    0.233340
Forward charges            0.217598
Name: Weight(kg), dtype: float64

## Merge Zone table

In [353]:
Zone=Zone.rename(columns = {'Zone' : 'XZone'})
Invoice_SKU_Courier_invoice_Zone = pd.merge(Invoice_SKU_Courier_invoice, Zone, how='inner', on=['Warehouse Pincode','Customer Pincode'])
Invoice_SKU_Courier_invoice_Zone.head(3)

,Order ID,SKU,Order Qty,Weight (g),Weight(kg),Weight Slab as per X,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),XZone
0,2001827036,8904223818706,1.0,127,0.127,0.5,1091122418320,1.6,121003,173213,b,Forward charges,117.9,e
1,2001827036,8904223819093,1.0,150,0.150,0.5,1091122418320,1.6,121003,173213,b,Forward charges,117.9,e
2,2001827036,8904223819109,1.0,100,0.100,0.5,1091122418320,1.6,121003,173213,b,Forward charges,117.9,e


## Slab multiply 0.5

In [354]:
Invoice_SKU_Courier_invoice_Zone['multi 0.5'] = (Invoice_SKU_Courier_invoice_Zone['Weight Slab as per X']/0.5)-1

In [355]:
Invoice_SKU_Courier_invoice_Zone['Type-XZone'] = Invoice_SKU_Courier_invoice_Zone['Type of Shipment'] + '-' + Invoice_SKU_Courier_invoice_Zone['XZone']
Invoice_SKU_Courier_invoice_Zone['Type-XZone']

0      Forward charges-e
1      Forward charges-e
2      Forward charges-e
3      Forward charges-e
4      Forward charges-e
             ...        
393    Forward charges-d
394    Forward charges-d
395    Forward charges-d
396    Forward charges-b
397    Forward charges-b
Name: Type-XZone, Length: 398, dtype: object

In [356]:
grouped = Invoice_SKU_Courier_invoice_Zone.groupby('Type-XZone')['Weight(kg)'].mean()
grouped 

Type-XZone
Forward and RTO charges-b    0.185235
Forward and RTO charges-d    0.265429
Forward and RTO charges-e    0.217200
Forward charges-b            0.213311
Forward charges-d            0.232898
Forward charges-e            0.183619
Name: Weight(kg), dtype: float64

In [357]:
import numpy as np

Invoice_SKU_Courier_invoice_Zone['XAmount'] = np.where(
    Invoice_SKU_Courier_invoice_Zone['Type-XZone'] == 'Forward charges-b',
    33 + (Invoice_SKU_Courier_invoice_Zone['multi 0.5'] * 28.3),
    np.where(
        Invoice_SKU_Courier_invoice_Zone['Type-XZone'] == 'Forward charges-d',
        45.4 + (Invoice_SKU_Courier_invoice_Zone['multi 0.5'] * 44.8),
    np.where(
        Invoice_SKU_Courier_invoice_Zone['Type-XZone'] == 'Forward charges-e',
        56.6 + (Invoice_SKU_Courier_invoice_Zone['multi 0.5'] * 55.5),
        np.where(
            Invoice_SKU_Courier_invoice_Zone['Type-XZone'] == 'Forward and RTO charges-b',
            20.5 + 33 + (Invoice_SKU_Courier_invoice_Zone['multi 0.5'] * 28.3 *2),
            np.where(
                Invoice_SKU_Courier_invoice_Zone['Type-XZone'] == 'Forward and RTO charges-d',
                41.3 + 45.4 + (Invoice_SKU_Courier_invoice_Zone['multi 0.5'] * 44.8 *2 ),
                np.where(
                    Invoice_SKU_Courier_invoice_Zone['Type-XZone'] == 'Forward and RTO charges-e',
                    50.7 + 56.6 + (Invoice_SKU_Courier_invoice_Zone['multi 0.5'] * 55.5 *2),
                    np.nan  # Default value if none of the conditions match
                )
            )
        )
    )
)
)

In [358]:
Invoice_SKU_Courier_invoice_Zone.head(3)

,Order ID,SKU,Order Qty,Weight (g),Weight(kg),Weight Slab as per X,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),XZone,multi 0.5,Type-XZone,XAmount
0,2001827036,8904223818706,1.0,127,0.127,0.5,1091122418320,1.6,121003,173213,b,Forward charges,117.9,e,0.0,Forward charges-e,56.6
1,2001827036,8904223819093,1.0,150,0.150,0.5,1091122418320,1.6,121003,173213,b,Forward charges,117.9,e,0.0,Forward charges-e,56.6
2,2001827036,8904223819109,1.0,100,0.100,0.5,1091122418320,1.6,121003,173213,b,Forward charges,117.9,e,0.0,Forward charges-e,56.6


## Check null 

In [359]:
Invoice_SKU_Courier_invoice_Zone['XAmount'].isnull().sum()

np.int64(0)

## Difference

In [360]:
Invoice_SKU_Courier_invoice_Zone['Diff_Amount'] = Invoice_SKU_Courier_invoice_Zone['XAmount'] - Invoice_SKU_Courier_invoice_Zone['Billing Amount (Rs.)']

In [361]:
Invoice_SKU_Courier_invoice_Zone.head(1)

,Order ID,SKU,Order Qty,Weight (g),Weight(kg),Weight Slab as per X,AWB Code,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),XZone,multi 0.5,Type-XZone,XAmount,Diff_Amount
0,2001827036,8904223818706,1.0,127,0.127,0.5,1091122418320,1.6,121003,173213,b,Forward charges,117.9,e,0.0,Forward charges-e,56.6,-61.3


## Sum Amount

In [362]:
Amountzero = Invoice_SKU_Courier_invoice_Zone.loc[
    Invoice_SKU_Courier_invoice_Zone['Diff_Amount'] == 0, 'XAmount'
].sum()


Amountgreaterzero = Invoice_SKU_Courier_invoice_Zone.loc[
    Invoice_SKU_Courier_invoice_Zone['Diff_Amount'] < 0, 'Diff_Amount'
].sum()


Amountlesszero = Invoice_SKU_Courier_invoice_Zone.loc[
    Invoice_SKU_Courier_invoice_Zone['Diff_Amount'] > 0, 'Diff_Amount'
].sum()

## Count Rows

In [363]:
countzero = (Invoice_SKU_Courier_invoice_Zone['Diff_Amount'] == 0).sum()
countgreaterzero = (Invoice_SKU_Courier_invoice_Zone['Diff_Amount'] < 0).sum()
countlesszero = (Invoice_SKU_Courier_invoice_Zone['Diff_Amount'] > 0).sum()

## Summary table - Result 

In [364]:
Result['Count'] = [countzero, countgreaterzero, countlesszero]
Result['Amount'] = [Amountzero, Amountgreaterzero, Amountlesszero]

In [365]:
Result

,Index,Count,Amount
0,Total Orders - Correctly Charged,12,507.6
1,Total Orders - Over Charged,377,-31508.1
2,Total Orders - Under Charged,9,165.2


## Exporting

In [367]:
with pd.ExcelWriter("Final_output.xlsx") as writer:
    Result.to_excel(writer, sheet_name="Summary", index=False)
    Invoice_SKU_Courier_invoice_Zone.to_excel(writer, sheet_name="Base Data", index=False)